In [1]:
TESTING_SUBSET = False  # ***** FOR TESTING ONLY !!!!!!!! ******

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:

    # Enable line-wrapping
    # from IPython.display import HTML, display

    # def set_css():
    #     display(HTML('''
    #         <style>
    #         pre {
    #             white-space: pre-wrap;
    #             }
    #         </style>
    #     '''))
    # get_ipython().events.register('pre_run_cell', set_css)


    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')

    %pip install transformers
    %pip install datasets
    %pip install pynvml

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
if IN_COLAB:
    # Enable line-wrapping
    from IPython.display import HTML, display

    def set_css():
        display(HTML('''
            <style>
            pre {
                white-space: pre-wrap;
                }
            </style>
        '''))
    get_ipython().events.register('pre_run_cell', set_css)


    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import datetime
import numpy as np
from sklearn.metrics import fbeta_score
from matplotlib import pyplot as plt
import math
import pandas as pd
import sys
import json
import time
import torch
from torch import nn
import random
#from tqdm.auto import tqdm
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import Dataset
import os
import torchsummary
from functools import partialmethod
from itertools import product

from transformers import AutoTokenizer, AutoModel
from dataclasses import dataclass
from transformers import PreTrainedTokenizer
from torch.utils.data import DataLoader

if IN_COLAB:
  sys.path.append('/content/drive/MyDrive/ML/lecr/code/')
from l_utils import *
from model_utils import *

#from model_defs import Topic_NN, Content_NN

%env CUBLAS_WORKSPACE_CONFIG=:16:8

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

env: CUBLAS_WORKSPACE_CONFIG=:16:8
cuda


In [5]:
if IN_COLAB:
    class CFG:
        MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
        DIRNAME_PROCESSED_DATA = '/content/drive/MyDrive/ML/lecr/data/'
        FILENAME_DATA_CONTENT_F = DIRNAME_PROCESSED_DATA + "content.f"
        FILENAME_DATA_TOPICS_F = DIRNAME_PROCESSED_DATA + "topics.f"
        FILENAME_DATA_CORRELATIONS_CSV = DIRNAME_PROCESSED_DATA + "correlations.csv"
        NUM_FOLDS = 5
        SAVE_MODEL_DIR = '/content/drive/MyDrive/ML/lecr/models/'
else:
    from cfg import *
topics_tokenized_pq_file = CFG.DIRNAME_PROCESSED_DATA + "topics_tokenized.pq"
contents_tokenized_pq_file = CFG.DIRNAME_PROCESSED_DATA + "contents_tokenized.pq"


In [6]:
!nvidia-smi

Fri Mar 24 19:55:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    44W / 400W |      3MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
# Split data into folds

# Feature engineering ... use same as the one used for bce-lightning

# Training for a Top-k metric ... see if this performs better than directly using minilm embeddings

In [8]:
def progress_bar_control(show_progress_bars = True):
    tqdm.__init__ = partialmethod(tqdm.__init__, disable=(not show_progress_bars))

In [9]:
contents_df = pd.read_feather(CFG.FILENAME_DATA_CONTENT_F)
topics_df   = pd.read_feather(CFG.FILENAME_DATA_TOPICS_F)
correlations_df =  pd.read_csv(CFG.FILENAME_DATA_CORRELATIONS_CSV)

In [10]:
contents_df.info()
topics_df.info()
correlations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154047 entries, 0 to 154046
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                154047 non-null  object
 1   title             154038 non-null  object
 2   description       89456 non-null   object
 3   kind              154047 non-null  object
 4   text              74035 non-null   object
 5   language          154047 non-null  object
 6   copyright_holder  71821 non-null   object
 7   license           74035 non-null   object
dtypes: object(8)
memory usage: 9.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76972 entries, 0 to 76971
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           76972 non-null  object
 1   title        76970 non-null  object
 2   description  34953 non-null  object
 3   channel      76972 non-null  object
 4   category     76972 no

In [11]:
topics_tokenized_df   = pd.read_parquet(topics_tokenized_pq_file)
contents_tokenized_df = pd.read_parquet(contents_tokenized_pq_file)

In [12]:
contents_tokenized_df.info()
topics_tokenized_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154047 entries, 0 to 154046
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   id                  154047 non-null  object
 1   c_0_input_ids       154047 non-null  object
 2   c_1_attention_mask  154047 non-null  object
dtypes: object(3)
memory usage: 3.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61517 entries, 0 to 61516
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  61517 non-null  object
 1   t_0_input_ids       61517 non-null  object
 2   t_1_attention_mask  61517 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [13]:
# train_df_orig, val_df_orig = create_train_val_split_pos_corr(topics_df, contents_df, correlations_df, CFG.NUM_FOLDS, min_train_perc = 80.0, use_topic_trees = True, random_seed = 3) # pick a random seed that gives a good split
train_df_orig, val_df_orig = create_train_val_split_pos_corr_from_kaggle(topics_df, correlations_df, num_splits = 4, val_fold = 0)

177290
102629
2023-03-24 19:55:22.883026 : Splitting non-source into 4 folds
fold
0.0    25667
1.0    25677
2.0    25663
3.0    25622
dtype: int64
fold
0.0    6070
1.0    4971
2.0    8493
3.0    5469
dtype: int64


In [14]:
# ***** FOR TESTING ONLY !!!!!!!! ******
if TESTING_SUBSET:
    train_df = train_df_orig[:1000]
    val_df = val_df_orig[:1000]
else:
    train_df = train_df_orig
    val_df = val_df_orig

In [15]:
train_df.info()
val_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254252 entries, 0 to 254251
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   t_id    254252 non-null  object
 1   c_id    254252 non-null  object
dtypes: object(2)
memory usage: 3.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25667 entries, 0 to 25666
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   t_id    25667 non-null  object
 1   c_id    25667 non-null  object
dtypes: object(2)
memory usage: 401.2+ KB


In [16]:
#train_df.loc[random.sample(range(0,len(train_df)),10)]
train_df.loc[train_df['t_id']=='t_f2872cd7562c']

,t_id,c_id
249849,t_f2872cd7562c,c_0227d2bbe4cf
249850,t_f2872cd7562c,c_1ec69bbecf6a
249851,t_f2872cd7562c,c_3eb7c9babbac
249852,t_f2872cd7562c,c_6c55a9120277
249853,t_f2872cd7562c,c_7a6f34a71b1a
249854,t_f2872cd7562c,c_9b8249e189bf
249855,t_f2872cd7562c,c_b5e5385e2d8b
249856,t_f2872cd7562c,c_c2481d9b0e88
249857,t_f2872cd7562c,c_c4218a848990


In [17]:
PARTITION_BATCH_SIZE = 350

In [18]:
train_tokenized_df = get_partitioned_data_for_pos_corr(train_df, topics_tokenized_df, contents_tokenized_df, batch_size = PARTITION_BATCH_SIZE, do_partitioning = False, sort = False)
# train_tokenized_df = get_labeled_data_for_pos_corr(train_df.copy(), topics_tokenized_df, contents_tokenized_df)

2023-03-24 19:55:23.215980 : Merging in topic and content data into the folds data ... 


In [19]:
train_tokenized_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254252 entries, 0 to 254251
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   t_0_input_ids       254252 non-null  object
 1   t_1_attention_mask  254252 non-null  object
 2   c_0_input_ids       254252 non-null  object
 3   c_1_attention_mask  254252 non-null  object
dtypes: object(4)
memory usage: 9.7+ MB


In [21]:
# Below is inplace transformation!
find_col_sep_indices(train_tokenized_df, sep_input_ids = [2,0], num_seperators = 3)

,t_0_input_ids,t_1_attention_mask,c_0_input_ids,c_1_attention_mask,t_0_input_ids_sep_indices,c_0_input_ids_sep_indices
0,"[0, 3858, 18229, 16317, 29, 6, 150652, 61056, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 7331, 14709, 1770, 6, 150652, 22885, 2, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10, 40, 64]","[8, 33, 35]"
1,"[0, 52920, 358, 15119, 27662, 12, 38273, 89, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 7331, 14709, 1770, 6, 150652, 22885, 2, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[9, 34, 64]","[8, 33, 35]"
2,"[0, 3858, 18229, 16317, 29, 6, 150652, 61056, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1721, 9282, 1551, 241140, 7474, 5341, 4967...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10, 40, 64]","[10, 55, 57]"
3,"[0, 52920, 358, 15119, 27662, 12, 38273, 89, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1721, 9282, 1551, 241140, 7474, 5341, 4967...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[9, 34, 64]","[10, 55, 57]"
4,"[0, 3858, 18229, 16317, 29, 6, 150652, 61056, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 2443, 1488, 14709, 6, 150652, 22885, 15, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10, 40, 64]","[19, 64, 64]"
...,...,...,...,...,...,...
254247,"[0, 3076, 454, 1062, 9016, 20, 540, 178149, 2,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 239, 199625, 2, 0, 2, 0, 2, 0, 2, 1, 1, 1,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[9, 11, 24]","[4, 6, 8]"
254248,"[0, 3076, 454, 1062, 9016, 20, 540, 178149, 2,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 97033, 7, 144, 14364, 2, 0, 2, 0, 2, 0, 2,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[9, 11, 24]","[6, 8, 10]"
254249,"[0, 3076, 454, 1062, 9016, 20, 540, 178149, 2,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 5599, 40, 42624, 8, 51, 178149, 2, 0, 2, 0...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[9, 11, 24]","[8, 10, 12]"
254250,"[0, 3076, 454, 1062, 9016, 20, 540, 178149, 2,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 44891, 14618, 12, 540, 178149, 2, 0, 2, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[9, 11, 24]","[7, 9, 11]"


In [22]:
train_tokenized_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254252 entries, 0 to 254251
Data columns (total 6 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   t_0_input_ids              254252 non-null  object
 1   t_1_attention_mask         254252 non-null  object
 2   c_0_input_ids              254252 non-null  object
 3   c_1_attention_mask         254252 non-null  object
 4   t_0_input_ids_sep_indices  254252 non-null  object
 5   c_0_input_ids_sep_indices  254252 non-null  object
dtypes: object(6)
memory usage: 13.6+ MB


In [23]:
train_tokenized_df[['t_0_input_ids_sep_indices']][0:10]

,t_0_input_ids_sep_indices
0,"[10, 40, 64]"
1,"[9, 34, 64]"
2,"[10, 40, 64]"
3,"[9, 34, 64]"
4,"[10, 40, 64]"
5,"[9, 34, 64]"
6,"[10, 40, 64]"
7,"[10, 20, 56]"
8,"[10, 28, 51]"
9,"[10, 32, 60]"


In [24]:
val_topics_tokenized_df, val_contents_tokenized_df, val_corr_df = get_pos_corr_subsets_for_binary_fold( val_df, contents_tokenized_df, topics_tokenized_df)

In [25]:
find_col_sep_indices(val_topics_tokenized_df, sep_input_ids = [2,0], num_seperators = 3)
find_col_sep_indices(val_contents_tokenized_df, sep_input_ids = [2,0], num_seperators = 3)

,index,id,c_0_input_ids,c_1_attention_mask,c_0_input_ids_sep_indices
0,0,c_00046806ad8a,"[0, 113307, 13, 6024, 9, 4803, 217, 101935, 2,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[9, 23, 25]"
1,1,c_000751f58836,"[0, 61765, 35066, 111, 130370, 2967, 15, 3355,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12, 35, 64]"
2,2,c_00153f28489c,"[0, 3731, 114602, 146, 20286, 2, 0, 2, 0, 2, 0...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[6, 8, 10]"
3,3,c_0016f92657ea,"[0, 9626, 144057, 3687, 34292, 2, 0, 6565, 452...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[6, 32, 64]"
4,4,c_0019d72212e6,"[0, 10934, 8945, 1861, 47, 2501, 432, 41975, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[16, 64, 64]"
...,...,...,...,...,...
17416,17416,c_ffe2bc80b2f0,"[0, 4764, 1909, 6, 41382, 7, 136, 86869, 116, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12, 24, 26]"
17417,17417,c_ffed65e76e39,"[0, 82077, 660, 12, 581, 38280, 136, 158, 7, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[15, 44, 64]"
17418,17418,c_ffed80ebd964,"[0, 29117, 209, 7, 136, 805, 7, 15, 157, 456, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[14, 32, 34]"
17419,17419,c_fff300566888,"[0, 39215, 22819, 4, 29805, 98, 70, 159, 28889...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[13, 19, 21]"


In [26]:
val_topics_tokenized_df.info()
val_contents_tokenized_df.info()
val_corr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6070 entries, 0 to 6069
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   index                      6070 non-null   int64 
 1   id                         6070 non-null   object
 2   t_0_input_ids              6070 non-null   object
 3   t_1_attention_mask         6070 non-null   object
 4   t_0_input_ids_sep_indices  6070 non-null   object
dtypes: int64(1), object(4)
memory usage: 237.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17421 entries, 0 to 17420
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   index                      17421 non-null  int64 
 1   id                         17421 non-null  object
 2   c_0_input_ids              17421 non-null  object
 3   c_1_attention_mask         17421 non-null  object
 4   c_0_input_ids_

In [27]:
# Below is the mean pooling value for an empty string('')
fill_value= [ 2.2692e-01,  8.1784e-02,  2.3543e-02, -3.0236e-02,  1.1576e-02,
        -8.4300e-02,  4.4282e-01,  1.8080e-01,  1.0470e-01,  5.3833e-02,
         7.7437e-02, -1.4961e-01,  1.2243e-01, -8.1676e-02, -4.6227e-02,
        -6.1527e-02,  3.3096e-02, -4.1630e-02, -8.8344e-02,  7.1950e-02,
         1.0894e-01, -7.9747e-02,  3.9635e-02, -2.4031e-02, -2.6754e-02,
        -1.0373e-01, -1.2516e-01,  4.3589e-02, -1.2622e-01, -4.5149e-01,
         9.8479e-02, -1.1660e-01,  1.8807e-01,  1.6847e-01, -1.3254e-02,
         3.5356e-01,  2.0549e-02,  5.1132e-02,  1.3689e-01,  5.7409e-02,
         4.7993e-02, -1.6225e-02,  5.7460e-02,  2.5289e-01,  1.8837e-02,
         1.0862e-01,  6.3384e-02,  1.0661e-01,  5.4518e-02,  1.0492e-01,
         1.2421e-01, -7.6383e-02, -1.3154e-01, -9.2597e-02,  7.3059e-02,
         1.5369e-01,  1.6029e-01,  8.9187e-02,  1.3891e-02,  1.4503e-01,
        -3.5868e-02,  8.0861e-02, -1.4321e+00,  4.4488e-01, -2.4948e-01,
        -1.7631e-01, -3.1910e-02, -5.7227e-02, -2.0145e-01,  2.3304e-01,
         1.6574e-01, -7.6594e-03,  1.3327e-02,  1.3708e-01,  4.2691e-02,
        -1.0119e-01,  8.7779e-02,  1.2072e-01,  4.7127e-02, -5.6786e-02,
         6.0016e-02,  8.9962e-02,  5.6046e-02, -6.7986e-02, -4.2279e-02,
         1.1394e-01,  2.1153e-02,  1.1632e-01,  5.6897e-02, -1.1929e-01,
        -4.2084e-02, -1.0021e-01,  3.5531e-01, -7.5849e-03,  2.5373e-01,
        -4.1627e-03,  9.1205e-02, -1.0644e-01,  9.2273e-02,  2.5044e+00,
        -5.8386e-02,  1.0841e-01, -3.7277e-02,  7.8029e-02, -3.0702e-02,
        -6.8038e-02,  6.1537e-02,  4.7724e-02,  5.5612e-03, -3.1199e-02,
        -4.8965e-02,  9.2526e-03, -3.8606e-02,  4.9563e-03,  2.4629e-02,
        -5.5974e-02, -1.4868e-01,  1.6004e-01,  7.8883e-02,  8.8697e-02,
        -9.3105e-02, -5.8991e-02, -3.6320e-02, -7.5871e-02,  2.2877e-02,
        -8.1515e-01,  2.1145e-01,  1.7188e-01, -1.1297e-01,  5.3246e-02,
         2.3842e-02,  1.1703e-01, -5.3804e-03,  1.5747e-01, -1.8410e-01,
        -7.9964e-02,  9.7024e-02,  2.4204e-01,  2.4760e-02, -1.6613e-01,
        -8.5000e-02,  2.4986e-01,  6.8966e-03,  7.9926e-02, -1.1779e-02,
        -9.7117e-02, -5.2436e-02, -1.7714e-01, -1.6264e-02,  1.1144e-01,
        -2.5390e-02, -2.8893e-02, -4.3197e-03,  1.1250e-02, -4.3276e-02,
        -1.3757e-01,  5.5093e-02, -4.4147e-02,  1.3151e-01,  2.3556e-01,
         5.5276e-03,  7.7791e-02,  2.2956e-02, -1.0002e-01,  1.1859e-02,
        -1.3781e-01, -1.0910e-01,  3.5769e-02,  1.6109e-01,  1.1679e-01,
         6.4749e-02, -1.9757e-02, -2.1249e-02,  8.0824e-02,  3.0256e-02,
         3.8728e-03,  2.0422e-02,  1.1680e-01, -6.6920e-02,  7.3023e-02,
         7.8251e-02, -2.7585e-02, -5.7218e-02,  1.7479e-01,  2.6265e-02,
         5.7760e-02,  5.9157e-03,  1.9767e-02,  2.0600e-01, -1.3334e-01,
         2.0944e-02, -1.0435e-01, -6.1183e-03, -2.6320e-02, -1.2500e-01,
        -6.1774e-02, -1.5096e-03,  8.4123e-02,  1.1914e-01,  4.6520e-02,
        -3.1477e-01, -1.3655e-01,  3.6399e-02, -1.9176e-01,  3.0008e-01,
        -9.3990e-02,  2.2608e-01, -4.6670e-03, -5.4418e-02,  2.7442e-02,
        -2.8967e-02, -8.9384e-02, -1.2748e-02,  6.2952e-02,  2.4998e-03,
        -5.5135e-02,  5.1330e-03,  1.0732e-01, -2.1539e-01, -2.5023e-03,
         4.0274e-02, -1.3176e-02,  8.1344e-02, -6.5001e-01,  1.7839e-01,
        -1.4987e-01, -1.1081e-01,  1.5927e-01, -4.7563e-02, -2.1222e-02,
         3.5939e-02,  6.1253e-02, -3.4650e-02, -1.2995e-03, -1.8612e-01,
        -1.0152e-01, -9.3176e-02,  1.0601e-02, -2.0797e-01, -2.1590e-02,
        -1.1457e-01, -8.9968e-02,  4.5233e-02,  2.2494e-01,  8.9541e-02,
        -1.1733e-01, -1.5098e-01,  4.1164e-01, -1.2247e-01,  4.6524e-01,
        -1.6075e-01, -1.2944e-01, -4.2891e-02,  5.1767e-02, -7.4368e-02,
         3.1824e-02, -1.6158e-01,  3.0682e-02,  2.3774e-02,  6.4392e-02,
         1.2988e-01, -1.5734e-01, -2.5637e-01, -1.3158e-01,  1.1741e-01,
        -1.2536e-01, -7.0073e-02,  4.5891e-01,  8.8894e-02,  9.4389e-02,
        -9.3856e-02, -2.3517e-02, -3.6313e-02, -6.8319e-02,  1.1310e-02,
        -9.5391e-02,  7.1369e-02, -1.7914e-01,  5.4749e-02,  1.0912e-01,
        -1.4087e-01, -3.3488e-01,  6.1394e-02,  1.6397e-01,  2.6653e-02,
        -5.0510e-02, -8.8065e-05,  1.2700e-01,  1.4438e-02,  1.0510e-01,
        -3.1389e-02,  2.0691e-01, -6.8276e-03,  3.1468e-03, -6.3190e-02,
        -6.8658e-02, -2.1973e-02,  3.0819e-02,  5.7650e-02,  2.2963e-01,
        -4.9901e-02, -2.5205e-01, -1.2389e-01, -1.0548e-02,  5.2336e-02,
        -3.9859e-02, -7.1185e-02,  3.0303e-02,  1.5215e-01, -1.0238e-01,
        -2.1264e-02,  7.3842e-02,  5.3625e-02,  4.2827e-02,  9.7442e-02,
         2.5675e-02, -2.0598e-02, -8.5624e-02,  6.3012e-03, -8.9776e-01,
        -4.5344e-02, -3.6481e-02, -1.7548e-01,  9.4196e-02,  2.4167e-01,
         4.9701e-02, -4.1320e-02, -1.6034e-01,  7.0406e-02, -2.8026e-02,
        -5.2292e-02,  2.1600e-02, -1.0621e-01, -6.8105e-02,  8.0148e-02,
         1.8454e-01, -2.1566e-01,  1.7252e-02,  2.1095e-02, -2.4297e-02,
         1.6635e-01,  1.7066e-01,  6.9920e-02, -5.6332e-04,  3.1908e-02,
         9.9579e-03, -3.5469e-02, -1.2809e-01,  1.0547e-01,  1.4812e-01,
         6.4424e-02, -8.2031e-01, -3.3207e-01, -3.8212e-02, -5.2067e-02,
         1.1385e-01,  8.5623e-03,  3.7771e-02,  1.3134e-01,  3.7512e-03,
         1.7528e-01,  6.9390e-02, -2.9216e-03,  1.4786e-01, -9.3166e-02,
         1.6176e-01, -6.5376e-02,  1.1545e-02,  1.5838e-02,  8.9128e-03,
        -1.3336e-02,  4.2593e-04, -1.8814e-03, -3.4167e-02,  1.6659e-01,
        -4.8261e-02,  4.2300e-02, -1.5586e-03, -1.6665e-01,  2.1690e-01,
         1.8591e-01, -9.9829e-02, -3.1076e-02,  7.4184e-02]
fill_value = torch.tensor(fill_value).unsqueeze(0).to(device)

In [28]:
def mean_pooling_with_token_embs(token_embeddings, attention_mask):
    """
    Average the output embeddings using the attention mask 
    to ignore certain tokens.
    """
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

def get_feature_embeddings(outputs, attention_mask, sep_indices, fill_value):
    # print(outputs.size())
    # print(attention_mask.size())
    embs = []
    for ex_idx in range(outputs.size(0)):
        start_idx = 0
        ex_embs = []
        for end_idx in sep_indices[ex_idx].tolist():
            if (start_idx < end_idx):
                outputs_slice = outputs[ex_idx:ex_idx+1,start_idx:end_idx,:]
                attention_mask_slice = attention_mask[ex_idx:ex_idx+1,start_idx:end_idx]
                # print(outputs_slice.size())
                # print(attention_mask_slice.size())
                ex_embs.append(mean_pooling_with_token_embs(outputs_slice, attention_mask_slice))
            else:
                ex_embs.append(fill_value)
            start_idx = end_idx
        embs.append(torch.cat(ex_embs, axis=1))
    return torch.cat(embs, axis=0)

class TransModel(torch.nn.Module):
    """
    Wrapper. Gets tokenized features and returns the output logits for whether the token and content are matching.
    """
    def __init__(self, trans_model_name):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(trans_model_name)
    
    # def get_transformer_output(self, input_ids, attention_mask):
    #     outputs = self.transformer(input_ids = input_ids, attention_mask=attention_mask)
    #     pooled_embeddings=self.mean_pooling_x(outputs[0], attention_mask)
    #     return pooled_embeddings

    # def forward(self, features):
    #     # Grab embeddings from transformer
    #     embs = self.get_transformer_output(
    #         features[:,0,:],
    #         features[:,1,:]
    #     )

    def forward(self, features, sep_indices):

        input_ids      = features[:,0,:]
        attention_mask = features[:,1,:]
        outputs = self.transformer(input_ids = input_ids, attention_mask=attention_mask)
        embs    = get_feature_embeddings(outputs[0], attention_mask, sep_indices, fill_value)
        return embs

from model_defs import LinearReLU

class TC_NN(nn.Module):
    def __init__(self, num_features, feature_emb_dim, output_emb_dim):
        super().__init__()

        self.num_features = num_features
        self.feature_emb_dim = feature_emb_dim
        
        neurons_tl1 = 256
        self.l1 = [LinearReLU(feature_emb_dim, neurons_tl1, add_batch_norm = True) for _ in range(num_features)]
        self.l1 = nn.ModuleList(self.l1)
        self.l2 = nn.Linear(neurons_tl1*num_features, output_emb_dim)
        # self.tl6 = nn.BatchNorm1d(emb_dim)

        self.dropout_30p = nn.Dropout(0.3)
        self.dropout_10p = nn.Dropout(0.1)
        
    def forward(self, x):
        x_temp = []
        for i in range(self.num_features):
            x_sub = x[:,i*self.feature_emb_dim:(i+1)*self.feature_emb_dim]
            x_sub = self.dropout_10p(x_sub)
            x_temp.append(self.l1[i](x_sub))
        x = torch.cat(x_temp, axis = 1)
        x = self.l2(x)

        return x


In [29]:
class CustomDS(torch.utils.data.Dataset):
    def __init__(self, tensor):
        self.topic               = tensor[0]
        self.topic_sep_indices   = tensor[1]
        self.content             = tensor[2]
        self.content_sep_indices = tensor[3]
            
    def __len__(self):
        return self.topic.size(dim=0)
    
    def __getitem__(self, item):
        
        topic               = self.topic[item]
        topic_sep_indices   = self.topic_sep_indices[item]
        content             = self.content[item]
        content_sep_indices = self.content_sep_indices[item]


        return topic, topic_sep_indices, content, content_sep_indices

In [30]:
t_cols = [col for col in train_tokenized_df.columns if col.startswith('t_') and not col.endswith('_sep_indices')]
t_col_sep_indices = [col for col in train_tokenized_df.columns if col.startswith('t_') and col.endswith('_sep_indices')]
c_cols = [col for col in train_tokenized_df.columns if col.startswith('c_') and not col.endswith('_sep_indices')]
c_col_sep_indices = [col for col in train_tokenized_df.columns if col.startswith('c_') and col.endswith('_sep_indices')]
train_ds = CustomDS(
                    (torch.tensor(train_tokenized_df[t_cols].values.tolist()),
                     torch.tensor(train_tokenized_df[t_col_sep_indices].values.tolist()).squeeze(1),
                     torch.tensor(train_tokenized_df[c_cols].values.tolist()),
                     torch.tensor(train_tokenized_df[c_col_sep_indices].values.tolist()).squeeze(1)
                     )
                    )


<ipython-input-30-2a3d8658da39>:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.tensor(train_tokenized_df[t_cols].values.tolist()),


In [31]:
train_ds.topic_sep_indices.size()

torch.Size([254252, 3])

In [32]:
val_data = (torch.tensor(val_topics_tokenized_df[t_cols].values.tolist()),
            torch.tensor(val_topics_tokenized_df[t_col_sep_indices].values.tolist()).squeeze(1),
            torch.tensor(val_contents_tokenized_df[c_cols].values.tolist()),
            torch.tensor(val_contents_tokenized_df[c_col_sep_indices].values.tolist()).squeeze(1)
            ,val_corr_df
            )

In [33]:
try: t_model
except NameError: t_model = None
if t_model is not None:
  del t_model
report_gpu()
BATCH_SIZE = 760 # 650 works for A100, 20 works good for training for TESTING_SUBSET
if TESTING_SUBSET:
    # 20 works good for training for TESTING_SUBSET
    BATCH_SIZE = 20
else:
    BATCH_SIZE = 1100
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers = 2)

GPU:0
process     185504 uses     1044.000 MB GPU memory


In [34]:
# Add option to load pre-existing model
LOAD_CHECKPOINT_MODEL = False
SAVE_BEST_MODEL = True
DRY_RUN = False
TRAIN_PATIENCE = 3
k_vals = [5,10, 50]
METRIC = 'recall'
METRIC_KEY = 10
MODEL_FILENAME_PREFIX = f'One_trans_MiniLM_MNRL_fp16_ml64_train_shuffled_parent_title_new_sep_batch_size_{BATCH_SIZE}'
SAVE_MODEL_DIR = CFG.SAVE_MODEL_DIR
NUM_FEATURES = 3
FEATURE_EMB_DIM = 384
OUTPUT_EMB_DIM = 512
trans_model = TransModel( trans_model_name = CFG.MODEL)
t_model = TC_NN(num_features = NUM_FEATURES, feature_emb_dim = FEATURE_EMB_DIM , output_emb_dim = OUTPUT_EMB_DIM)
c_model = TC_NN(num_features = NUM_FEATURES, feature_emb_dim = FEATURE_EMB_DIM , output_emb_dim = OUTPUT_EMB_DIM)
optimizer = torch.optim.Adam((list(trans_model.parameters()) + list(t_model.parameters()) + list(c_model.parameters())),
                              lr=2e-4,
                             #weight_decay = 1e-4 # L2 regularization, 1e-4 is a good value
                             )
model_dict = {'trans_model': trans_model, 't_model': t_model, 'c_model': c_model, 'optimizer': optimizer}
if LOAD_CHECKPOINT_MODEL:
    MODEL_FILENAME = SAVE_MODEL_DIR + 'One_trans_MiniLM_MNRL_fp16_ml64_train_shuffled_parent_title_new_sep_batch_size_1100.last_model.pth'
    # t_model, optimizer = load_model(t_model, optimizer = None, filename = MODEL_FILENAME)
    # We'll use a fresh optimizer since we are loading ArcFace pre-trained model.
    # optimizer = torch.optim.Adam((list(t_model.parameters())),
    #                           lr=2e-4,
    #                          #weight_decay = 1e-4 # L2 regularization, 1e-4 is a good value
    #                          )
    # train_scores = []
    # val_scores = []
    # epoch_count = 0
    # MODEL_FILENAME = None
    load_model( model_dict, filename = MODEL_FILENAME)
    train_data_dict = load_dict(SAVE_MODEL_DIR + f'{MODEL_FILENAME_PREFIX}.train.txt')
    train_scores, val_scores = train_data_dict['train_scores'], train_data_dict['val_scores']
    epoch_count = len(val_scores)
else:
    train_scores = []
    val_scores = []
    epoch_count = 0
    MODEL_FILENAME = None

trans_model = trans_model.to(device)
t_model = t_model.to(device)
c_model = c_model.to(device)
# criterion = nn.BCEWithLogitsLoss()
criterion = MultipleNegativesRankingLoss()



In [ ]:
NUM_EPOCHS = 15

DISABLE_TQDM = False
if (DISABLE_TQDM):
    tqdm.__init__ = partialmethod(tqdm.__init__, disable=DISABLE_TQDM)

scaler = torch.cuda.amp.GradScaler()

pt('Starting Training')
trans_model.train()
t_model.train()
c_model.train()
#for epoch in tqdm(range(EPOCHS), desc="Epochs"):
for epoch in range(NUM_EPOCHS):
    # We set the below in case it is frozen while doing eval
    pt(f"***** Epoch {epoch_count} *****")
    train_loss = 0
    #train_acc = 0
    #train_f2_score = 0
    for (topic, t_sep_indices, content, c_sep_indices) in tqdm(train_loader, desc="Training"):
        topic   = topic.to(device)
        content = content.to(device)

        optimizer.zero_grad()

        with torch.autocast(device_type='cuda', dtype=torch.float16):
            t_emb = trans_model(topic, t_sep_indices)
            c_emb = trans_model(content, c_sep_indices)

            t_emb = t_model(t_emb)
            c_emb = c_model(c_emb)
            
            loss = criterion(t_emb, c_emb)

        # loss.backward()
        # optimizer.step()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
       
        train_loss += loss.item()
    train_loss = round(train_loss/len(train_loader), 5)
    pt(f'Train : | Loss: {train_loss:.5f} ')
    train_scores.append({'loss':train_loss})
    recalls  = compute_recall_at_k_for_one_trans_two_towers(trans_model, t_model, c_model, device, val_data[0], val_data[1], val_data[2], val_data[3], val_data[4], k_vals = k_vals, batch_size = BATCH_SIZE)
    recall_str = '\nValidation:  ' + ''.join([f'| Recall@{k}: {recalls[k]:.3f}' for k in k_vals])
    pt(recall_str)
    val_scores.append({'recall':recalls})
    if SAVE_BEST_MODEL:
        MODEL_FILENAME = save_best_model( 
                                        model_dict = model_dict,
                                        old_model_filename = MODEL_FILENAME,
                                        model_filename_prefix = MODEL_FILENAME_PREFIX,
                                        model_path = SAVE_MODEL_DIR,
                                        save_model_fn = save_model,
                                        train_scores = train_scores,
                                        val_scores = val_scores, 
                                        metric = METRIC, metric_key = METRIC_KEY,
                                        dry_run = DRY_RUN
                                        )
    save_model(model_dict, SAVE_MODEL_DIR + f"{MODEL_FILENAME_PREFIX}.last_model.pth")
    epoch_count += 1
    save_dict({'train_scores':train_scores, 'val_scores':val_scores}, SAVE_MODEL_DIR + f'{MODEL_FILENAME_PREFIX}.train.txt')
    if not check_improvement(val_scores, metric = METRIC, metric_key = METRIC_KEY, patience = TRAIN_PATIENCE):
        pt(f'Exiting training since no improvement in the last {TRAIN_PATIENCE} epochs')
        break


2023-03-24 19:55:47.664340 : Starting Training
2023-03-24 19:55:47.665770 : ***** Epoch 0 *****


Training: 100%|██████████| 232/232 [13:00<00:00,  3.36s/it]


2023-03-24 20:08:47.736139 : Train : | Loss: 1.62606 


100%|██████████| 6/6 [00:03<00:00,  1.79it/s]


2023-03-24 20:09:00.592119 : 
Validation:  | Recall@5: 0.422| Recall@10: 0.555| Recall@50: 0.791
2023-03-24 20:09:00.592212 : Saving /content/drive/MyDrive/ML/lecr/models/One_trans_MiniLM_MNRL_fp16_ml64_train_shuffled_parent_title_new_sep_batch_size_1100_epoch_0_recall_10_0.555.pth
2023-03-24 20:09:10.021139 : ***** Epoch 1 *****


...

Training: 100%|██████████| 232/232 [13:00<00:00,  3.36s/it]


2023-03-24 22:48:55.438807 : Train : | Loss: 0.23372 


100%|██████████| 6/6 [00:03<00:00,  1.79it/s]


2023-03-24 22:49:08.199381 : 
Validation:  | Recall@5: 0.610| Recall@10: 0.751| Recall@50: 0.881
2023-03-24 22:49:08.199475 : Deleting /content/drive/MyDrive/ML/lecr/models/One_trans_MiniLM_MNRL_fp16_ml64_train_shuffled_parent_title_new_sep_batch_size_1100_epoch_11_recall_10_0.748.pth
2023-03-24 22:49:08.201685 : Saving /content/drive/MyDrive/ML/lecr/models/One_trans_MiniLM_MNRL_fp16_ml64_train_shuffled_parent_title_new_sep_batch_size_1100_epoch_12_recall_10_0.751.pth
2023-03-24 22:49:16.266722 : ***** Epoch 13 *****


Training:   1%|          | 2/232 [00:07<13:33,  3.54s/it]

In [ ]:
if IN_COLAB:
    from google.colab import runtime
    runtime.unassign()